# **Sample Neural Network**

based on the work of Hui et al.

# Libraries Used
- Flux is the main library as it holds the proper methods to do everything that needs to be done.
- DataLoader is a nice way to create mini batches from large collections of data and make the training process more efficient.
- CUDA is used to enable the utilization of the computer's GPU and speed up the whole training process (#PCMasterRace)
- Plots is used for visualization of the data
- Images is used to load and extract data from the airfoils
- BSON is used for saving/loading the Neural Network
- Dates is self explanatory 📆📆📆
- FileIO is used to load the data.

In [5]:
#Initially are called the basic libraries to use in order to work
using Flux
using Flux: Data.DataLoader
using Statistics
using CUDA
using Plots
using Flux: throttle
using BSON: @save
using BSON
using Dates: now
using Images
using FileIO
using LaTeXStrings
using DelimitedFiles
gr()

Plots.GRBackend()

# Input 
As an input is used an 32x32 px image containing all the geometrical characteristics of the airfoil in the form Signed Distance Function.

As importing an image and then taking its rgb values is not optimal a method, the data would be stored directly in the format of (Width,Heigth,Channel,Batch). So instead of images the data shall be stored in (32,32,3,N) arrays where N the Number of Speciments.

For evaluating the output of the Neural Network an array of (L,1) points is expected where L is the number of points of interest in the airfoil geometry. So the data for training and validation must be packed in a format of (L,N) where N is the BatchSize or Number of Speciments.

- [ ] **I have to create a subfolder for the training and validation sets**


~~The data shall be entered as an image that needs to be altered to the format WHCN (width, height, (color??)channel, batch#). So that implies that the input image used by the NN it is 40x40 px input but must be brought to 40x40x3xBatchSize.~~(old)

## The input data are needed in WHCN format.
~~However that may be a little inconvient during the data creation as addind an extra dimension may be confusing than working with a simple 3D Array to store the data. The RGB values however can ba stored as an Array inside each 3D cell of the Array.Then unsqueeze method can be used to extract this one extra dimension we need.~~( that's not the case)

**In Contrast to the before mentioned idea, it may be actually way more practical to work with 3D matrices that contain imformation about 1 image only**. Then in order to create the training data list we can simply push!() them into an array.

**4/13/21 Actually, what happened was that the image was read as Array Containing Tuples of RGBA values.** Then using channelview() the data were resized as an [Channel,Width,Height] Array and the i remapped them to an WHCN type array. 
>ez pz

**Flux.unsqueeze(ImageData(Expects Array with values),channels) just adds a dimension**. Practical for when the data need to be expressed as an N-dim array yet we have N-1 dimensions (ie. Grayscale Image not stored as (pxw,pxh,ch) but (pxw,pxh). However if the data is already in this format we have no use for this method. 

For the input data they will be extracted as an Nx1 array, where N the data points along the airfoil. And then similarly to the input data they will need to be concatenated into a Batch array
 - [x] **I HAVE TO SEE HOW I AM ACTUALLY GOING TO STORE THE DATA. WHCN OR 2D-ARRAY PER PHOTO WHERE THE RGB VALUES ARE AN ARRAY OR TUPLE IN EACH CELL**
 
-> Solution is found below 🍆



In [2]:
#Images loading segment
# The following arrays have the proper format in order to be fed in the Neural Network
# so any IO arrays have to be formatted to these types and sizes

io = open("/home/freshstart/DiplomaThesisData/DIRS","r") 
dirs=readlines(io)
close(io)

airfoils_tr = Array{String}(undef,0)
airfoils_ts = Array{String}(undef,0)

for dir in dirs 
    list = split(dir,"/")
    if list[5] == "train" 
        push!(airfoils_tr,dir)
    elseif list[5] == "test"
        push!(airfoils_ts,dir)
    else
        println("There was some kind of error")
        for i in 1:size(list)[1]
            println("$i --> $list[i]")
        end
    end
end

#TRAINING IMAGES LOADING
# airfoils_tr = readdir("C://AA_NeuralNetwork_ImagesFolder//",join =true)#WINDOWS TEST
# airfoils_tr = readdir("~/DiplomaThesisData",join =true)
N_tr = size(airfoils_tr)[1] #training batch dynamically allocated to the training batch size
N_ts = size(airfoils_ts)[1] #testing batch dynamically allocated to the training batch size
L = 49 #points of interest on the airfoil geometry per side

train_images = Array{Float32}(undef,(32,32,3,N_tr));
Points_Cp_up_train = Array{Float32}(undef,(L,N_tr));
Points_Cp_down_train = Array{Float32}(undef,(L,N_tr));

test_images = Array{Float32}(undef,(32,32,3,N_ts));
Points_Cp_up_test = Array{Float32}(undef,(L,N_ts));
Points_Cp_down_test = Array{Float32}(undef,(L,N_ts));



N = 1
@time for airfoil in airfoils_tr
    #--- image loading-----
    image = load(airfoil*"RAE_var.png") 
    image = channelview(image) #CWH not WHC ie. 3X32X32
    Channels,Width,Height=size(image[:,:,:])
    for C = 1:Channels-1,W = 1:Width,H = 1:Height
        train_images[W,H,C,N] = image[C,W,H] #Switching from CWH to WHCN
    end
    #-----------------------
    #----- Cp data loading--
    
    file = open(airfoil*"Cp_RAE_up_var.dat")
    lines=readlines(file);
    num=[]
    for i in 1:length(lines)
        push!(num,parse(Float32, lines[i]))
    end
    Points_Cp_up_train[:,N]=num    
    close(file)
    
    file = open(airfoil*"Cp_RAE_down_var.dat")
    lines=readlines(file);
    num=[]
    for i in 1:length(lines)
        push!(num,parse(Float32, lines[i]))
    end
    Points_Cp_down_train[:,N]=num    
    close(file)

    N +=1
#     break
end

N = 1
@time for airfoil in airfoils_ts
    #--- image loading-----
    image = load(airfoil*"RAE_var.png") 
    image = channelview(image) #CWH not WHC ie. 3X32X32
    Channels,Width,Height=size(image[:,:,:])
    for C = 1:Channels-1,W = 1:Width,H = 1:Height
        test_images[W,H,C,N] = image[C,W,H] #Switching from CWH to WHCN
    end
    #-----------------------
    #----- Cp data loading--
    
    file = open(airfoil*"Cp_RAE_up_var.dat")
    lines=readlines(file);
    num=[]
    for i in 1:length(lines)
        push!(num,parse(Float32, lines[i]))
    end
    Points_Cp_up_test[:,N]=num    
    close(file)
    
    file = open(airfoil*"Cp_RAE_down_var.dat")
    lines=readlines(file);
    num=[]
    for i in 1:length(lines)
        push!(num,parse(Float32, lines[i]))
    end
    Points_Cp_down_test[:,N]=num    
    close(file)

    N +=1
#     break
end

#-- Converting the data arrays to Cuda Arrays to be processed by the gpu
train_images = train_images |> gpu;
Points_Cp_up_train = Points_Cp_up_train |> gpu;
Points_Cp_down_train = Points_Cp_down_train |> gpu;
    
test_images = test_images |> gpu
Points_Cp_up_test = Points_Cp_up_test |> gpu;
Points_Cp_down_test = Points_Cp_down_test |> gpu;


  2.395741 seconds (7.96 M allocations: 243.557 MiB, 1.85% gc time, 8.77% compilation time)
  0.640519 seconds (3.43 M allocations: 89.287 MiB, 2.72% gc time)


## 9/4/21 Progress made. 
**The data can be stored as an list or array of 3D matrices containing the image information about each training item**. The using Flux.stack(data,4) i can concatenate the data along a new 4th axis and make them compatible with what is expected as input from the Neural Net. SO there is no need to load the image data in WHCN format as i can simply work with easier to manipulate 3D matrices.

In [32]:
#Batch creation for training and testing of the Neural Network
#No information in the bibliography, so i hypothesized that splitting the data in 3 parts 
#may be usefull.However, even 200 may be a big batch to process
up_train_data=DataLoader((train_images,Points_Cp_up_train),batchsize=025) |>gpu;
up_test_data=DataLoader((test_images,Points_Cp_up_test),batchsize=025) |>gpu;
down_train_data=DataLoader((train_images,Points_Cp_down_train),batchsize=025) |>gpu;
down_test_data=DataLoader((test_images,Points_Cp_down_test),batchsize=025) |>gpu;

# Model Creation

The most efficient model proposed by Hui et al. is structured like this :

- Convolutional Layer
- Batch Normalization Layer with ReLU activation Function 
- A simple Fully Connected Layer to extract the evaluated data

After some minor reverse engineering, turns out the following architecture was proposed:

In [33]:
CNN_C5_up = Chain(
Conv((5,5),3=>20,pad=2,stride=1),
BatchNorm(20,relu),
Conv((5,5),20=>40,pad=1,stride=1),
BatchNorm(40,relu),
Conv((5,5),40=>60,pad=1,stride=1),
BatchNorm(60,relu),
Conv((5,5),60=>80,pad=1,stride=1),
BatchNorm(80,relu),
Conv((5,5),80=>100,pad=1,stride=2),
BatchNorm(100,relu),
flatten,
Dense(14400,49) #14400 is manually calculated
) |>gpu

CNN_C5_down = Chain(
Conv((5,5),3=>20,pad=2,stride=1),
BatchNorm(20,relu),
Conv((5,5),20=>40,pad=1,stride=1),
BatchNorm(40,relu),
Conv((5,5),40=>60,pad=1,stride=1),
BatchNorm(60,relu),
Conv((5,5),60=>80,pad=1,stride=1),
BatchNorm(80,relu),
Conv((5,5),80=>100,pad=1,stride=2),
BatchNorm(100,relu),
flatten,
Dense(14400,49) #14400 is manually calculated
) |>gpu

ps_up = params(CNN_C5_up) 
ps_down = params(CNN_C5_down) ;

# Loss Function, Optimizer, Accuracy


In [2]:
loss_u(x,y)=Flux.mse(CNN_C5_up(x),y); #can and will be used for accuracy too
loss_d(x,y)=Flux.mse(CNN_C5_down(x),y); #can and will be used for accuracy too
opt = ADAM(0.0001);

# Training methods

I created 2 custom training and evaluating methods that are not that advanced, but they can actually work with GPU processing not like the default train!().

GpuValid() method is based on the respective training method, but instead it evaluates the accuracy of the Neural Net. Its use is optional.

In [16]:
function GpuTrain(loss,ps,data,opt,accuracy)
    acc = []
    for batch in data
        gs=gradient(ps)do
            train_loss=loss(batch...)
        return train_loss
        end
        Flux.update!(opt,ps,gs)
        push!(acc,accuracy(batch...))
    end
    mean_accuracy = mean(acc)
#     println("Mean error was $mean_accuracy ")
    return mean_accuracy
end
        

GpuTrain (generic function with 1 method)

In [17]:
function GpuValid(data,accuracy)
    acc=[]
    for batch in data
        push!(acc,accuracy(batch...))
    end
    mean_accuracy=mean(acc)
    return mean_accuracy
end

GpuValid (generic function with 1 method)

In [22]:
#a nifty function for time log
date = () -> (
    date =replace(replace(string(now()),"."=>"_"),":"=>"-");
    return date)
savemodel = (str, md, ps)->(@save "/home/freshstart/DiplomaThesisData/NeuralNetSaves/model"*str*".bson" md ps) 
saveModels=(state::Bool) -> (
    up = CNN_C5_up |> cpu;
    down = CNN_C5_down |> cpu;
    psup = ps_up ;
    psdown = ps_down ;
    if state
            savemodel("-"*date()*"-up",up,psup)
            savemodel("-"*date()*"-down",down,psdown)
        # end;
    else
        savemodel("-up",up,psup)
        savemodel("-down",down,psdown)
    end
)
loadModels = (path;top="model-up.bson",down="model-down.bson") -> (
    filename = path*top;
    up = BSON.load(filename);
    filename = path*down;
    down = BSON.load(filename);
    return up,down)

loadModelsGPU = (path;top="model-up.bson",down="model-down.bson") -> (
    filename = path*top;
    up = BSON.load(filename) ;
    model_up = up ;
    filename = path*down;
    down = BSON.load(filename) ;
    model_down = down;    
    return model_up,model_down)


#34 (generic function with 1 method)

# Training Point

- [x] **Saving the model state and loading needs to be added** 

In [ ]:
acc_up=[];vld_up=[]
acc_down=[];vld_down=[]
time=Array{Float64}(undef,(1,4))

end_t = 40
for i in 1:end_t
    Flux.@epochs 100 (
    ta = @elapsed push!(acc_up,GpuTrain(loss_u,ps_up,up_train_data,opt,loss_u) |>gpu) ;
    tb = @elapsed push!(vld_up,GpuValid(up_test_data,loss_u) |>gpu);#|>gpu;
    tc = @elapsed push!(acc_down,GpuTrain(loss_d,ps_down,down_train_data,opt,loss_d) |>gpu) ;#|>gpu;
    td = @elapsed push!(vld_down,GpuValid(down_test_data,loss_d) |>gpu);#|>gpu;
    time = vcat(time,[ta tb tc td]))
    println("$i out  of $(end_t*100) epochs")
    saveModels(true)
end

saveModels(false)
time = time[2:end,:];


In [35]:
#Save the training and testing data to use later without needing to retrain the net
data = hcat(acc_up,acc_down,vld_up,vld_down,time)

filename = "/home/freshstart/DiplomaThesisData/NeuralNetSaves/MSEsTime.dat"
text = "acc_up acc_down vld_up vld_down time\n"

for i in 1:size(data)[1], j in 1:size(data)[2]
    text *= "$(data[i,j]) "
    if j == size(data)[2]
        text = chop(text)
        text *= "\n"
    end
end

open(filename,"w") do file
    write(file,text)
end
    


398892

### Data visualization of the error and traing-testing times during the training session

In [36]:
plotly()
col = [RGB(i) for i in [0x00ab9f,0x7409b7,0xd2cf00,0xff6f00] ] # mah precious colors

plot(acc_up,label="training loss upper face")
plot!(vld_up,label="verification loss upper face")
plot!(acc_down,label="training loss bottom face")
plot!(vld_down,label="verification loss bottom face")
plot!( yscale=:log10,
    minorgrid =true,
    minortick=true,
    xlabel= "Epochs",
    ylabel= "Mean Square Error }",
    title="Training and verification loss",
    legend= :topright,size=(1000,9/16*1000))
ylims!((0.9*minimum((minimum(acc_up),minimum(acc_down),minimum(vld_up),minimum(vld_down))),1e-2))
display(plot!())   



labels=["training top side","validation top side","training bottom side","validation bottom side"]
plot(title="Neural Networks' timelog",xlabel = "Epochs",ylabel="Time [ms]"
        ,minorticks = true,)
for i in 1:4
    plot!(time[2:end,i]*1000,label = labels[i])
end
display(plot!())

# Neural Network Loading


In [23]:
a = loadModelsGPU("/home/freshstart/DiplomaThesisData/NeuralNetSaves_050/")
CNN_C5_up = a[1][:md] |> gpu
CNN_C5_down = a[2][:md] |> gpu
# a = BSON.load("/home/freshstart/DiplomaThesisData/NeuralNetSaves_050/model-up.bson" )[:md]
# println(a)

Chain(
  Conv((5, 5), 3 => 20, pad=2),         # 1_520 parameters
  BatchNorm(20, relu),                  # 40 parameters, plus 40
  Conv((5, 5), 20 => 40, pad=1),        # 20_040 parameters
  BatchNorm(40, relu),                  # 80 parameters, plus 80
  Conv((5, 5), 40 => 60, pad=1),        # 60_060 parameters
  BatchNorm(60, relu),                  # 120 parameters, plus 120
  Conv((5, 5), 60 => 80, pad=1),        # 120_080 parameters
  BatchNorm(80, relu),                  # 160 parameters, plus 160
  Conv((5, 5), 80 => 100, pad=1, stride=2),  # 200_100 parameters
  BatchNorm(100, relu),                 # 200 parameters, plus 200
  Flux.flatten,
  Dense(14400, 49),                     # 705_649 parameters
)         # Total: 22 trainable arrays, 1_108_049 parameters,
          # plus 10 non-trainable, 600 parameters, summarysize 4.328 KiB.

# Neural Network testing single input speed and precision

In [24]:
function calc_one(image)
    image = image |> gpu
    y_up=CNN_C5_up(image)
    y_down=CNN_C5_down(image)
    y_up = y_up |> cpu
    y_down = y_down |> cpu
    return y_up,y_down
end

calc_one (generic function with 1 method)

In [20]:
# FATHER FOIL RAE-2822 Loading for testing
FF=Array{Float32}(undef,(32,32,3,1))
#--- image loading-----
beep = "/home/freshstart/DiplomaThesisData/RAE_2822_baseline/"
image = load(beep*"RAE_var.png") 
image = channelview(image) #CWH not WHC ie. 3X32X32
Channels,Width,Height=size(image[:,:,:])
for C = 1:Channels-1,W = 1:Width,H = 1:Height
    FF[W,H,C,1] = image[C,W,H] #Switching from CWH to WHCN
end
#-----------------------

#----- Cp data loading--
    
file = open(beep*"Cp_RAE_up_var.dat")
lines=readlines(file);
CP_up=[]
for i in 1:length(lines)
    push!(CP_up,parse(Float32, lines[i]))
end    
close(file)
    
file = open(beep*"Cp_RAE_down_var.dat")
lines=readlines(file);
CP_down=[]
for i in 1:length(lines)
   push!(CP_down,parse(Float32, lines[i]))
end
close(file)

In [25]:
@time up,down=calc_one(FF)

 10.318258 seconds (18.49 M allocations: 1.037 GiB, 2.26% gc time, 0.11% compilation time)


(Float32[0.5434662; -0.88576514; … ; 0.13575114; 0.13190606], Float32[0.5363214; 0.93138427; … ; 0.22239965; 0.13380218])

In [26]:
# @time println(GpuValid(up_test_data,loss_u) |>gpu)
# @time println(GpuValid(down_test_data,loss_d) |>gpu)
bench = []

@time for i in 1000
t=@elapsed up,down=calc_one(FF)
push!(bench,t)
end

sum = 0
for i in bench
    sum+=i
end

sum/length(bench)



  0.001689 seconds (1.10 k allocations: 49.453 KiB)


0.001686579

In [39]:
pgfplotsx()

Plots.PGFPlotsXBackend()

In [41]:
x = [0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.5,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.58,0.59,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.96,0.97,0.98,0.99,1]
col = [RGB(i) for i in [0x00ab9f,0x7409b7,0xd2cf00,0xff6f00] ] # mah precious colors
# pgfplotsx()
plot(title = "Cp distribution between CFD simulation and Neural Net Prediction", yflip = true)
plot!(x,up,label = "CNN prediction distribution",c=col[1])
plot!(x,down,label="",c=col[1])
plot!(x,CP_up,label = "CFD simulation distribution",c=col[4])
plot!(x,CP_down,label="",c=col[4])

┌ Warning: failed to find logfile at "/tmp/jl_rP5viu.log"
└ @ PGFPlotsX /home/freshstart/.julia/packages/PGFPlotsX/1SEv2/src/build.jl:54


ErrorException: The latex command `lualatex jl_rP5viu.tex` failed

### Data visualization after checking a single foil

#### Foil Cp plots

path = "/home/freshstart/DiplomaThesisData/Paper-MaPFlow_chart.csv"

# file = open(path,"r")
data = readdlm(path,',')

xaxis = [data[:,15]]
paper_up=[data[:,1],data[:,2]]
paper_down=[data[:,3],data[:,4]]
MF=[data[:,5],data[:,7],data[:,6]]
Nasa=[data[:,8],data[:,10],data[:,9]]
exp_up=[data[:,13],data[:,14]]
exp_down=[data[:,11],data[:,12]]

str_el = (data) ->(
    for i ∈ 1:size(data)[1]; 
        pos=[];
        for j ∈ 1:size(data[i])[1];
            if !((typeof(data[i][j])==Float64)|(typeof(data[i][j])==Int64));
                push!(pos,j);
            end;
        end;
        pos
        for p ∈ 1:size(pos)[1];            
            deleteat!(data[i],pos[size(pos)[1]-p+1]);
        end;
    end;
    return data;
)

paper_up=str_el(paper_up)
paper_down=str_el(paper_down)
MF=str_el(MF)
Nasa=str_el(Nasa)
exp_up=str_el(exp_up)
exp_down=str_el(exp_down)    
xaxis =str_el(xaxis)[1];

gr() 

plot(xaxis,up,label="top side NN",color=:aquamarine4,w=2,line=:dash)
plot!(xaxis,down,label="bottom side NN",color=:royalblue,w=2,line=:solid)
plot!(xaxis,CP_up,label="top side MapFlow",color=:coral,w=2,line=:dashdot)
plot!(xaxis,CP_down,label="bottom side MapFlow",color=:red,w=2,line=:solid)
# plot!(Nasa[1],Nasa[2],label="top side MapFlow")
# plot!(Nasa[1],Nasa[3],label="bottom side MapFlow")
plot!(yflip=true,
      ylabel = L"Cp",
      xlabel = L"X\ axis\ normalized",
      title = L"Cp\ Distribution\ ",size=(750,9/16*750))

deleteat!(acc_up,1:6000)
deleteat!(vld_up,1:6000)
deleteat!(acc_down,1:6000)
deleteat!(vld_down,1:6000)
    

gr() 
c= [0x003f5c,0x7b5295,0xf05775,0xffa800]
width = 1000
s=(width,9/16*width)

var="255"
plot(xaxis,CP_up,label="RAE-2822 top side",color=RGB(c[1]),w=3,line=:dash)
plot!(xaxis,CP_down,label="RAE-2822 bottom side",color=RGB(c[1]),w=3,line=:solid)
plot!(xaxis,Points_Cp_up_test[:,parse(Int16,var)+1] |> cpu ,label="Var#1$var top side",color=RGB(c[4]),w=3,line=:dashdot)
plot!(xaxis,Points_Cp_down_test[:,parse(Int16,var)+1] |> cpu,label="Var#1$var bottom side MapFlow",color=RGB(c[4]),w=3,line=:solid)
# plot!(Nasa[1],Nasa[2],label="top side MapFlow")
# plot!(Nasa[1],Nasa[3],label="bottom side MapFlow")
plot!(yflip=true,
      ylabel = L"Cp",
      xlabel = L"X\ axis\ normalized",
      title = L"Cp\ Distribution\ ",size=s)